In [1]:
import numpy as np
from mealpy import PSO, TS, SA, ACOR, GA, Problem, PermutationVar

# Define the input data
M = 2
N = 5
processing_times = [
    [1, 68],
    [59, 25],
    [65, 46],
    [28, 28],
    [92, 93]
]

class JSSP(Problem):
    def __init__(self, bounds, minmax, name="JSSP", processing_times=None, **kwargs):
        self.name = name
        self.processing_times = processing_times
        super().__init__(bounds, minmax, **kwargs)
    
    # def obj_func(self, solution):
    #     solution = self.decode_solution(solution)["per"]
    #     total_completion_time = 0
    #     for machine in range(M):
    #         machine_time = 0
    #         for job in solution:
    #             machine_time += processing_times[job][machine]
    #             total_completion_time += machine_time
    #     # print(solution)
    #     # print(total_completion_time)
    #     return total_completion_time

    def obj_func(self, solution):
        solution = self.decode_solution(solution)["per"]
        num_jobs = len(solution)
        num_machines = len(processing_times[0])
        
        # Initialize completion times matrix
        completion_times = [[0] * num_machines for _ in range(num_jobs)]
        total_completion_time = 0

        for job_index in range(num_jobs):
            job = solution[job_index]

            for machine in range(num_machines):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == num_machines - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time
    
    def amend_position(self, solution):
        unique_jobs = np.unique(solution)
        if len(unique_jobs) == len(solution):
            # The solution is already a valid permutation
            return solution
        else:
            # If it's not, we reconstruct a valid permutation
            # This scenario might be rare depending on how the solution is generated or modified
            n = len(solution)
            valid_permutation = np.random.permutation(n)
            return valid_permutation.tolist()


## Create an instance of DOP class
problem_jssp = JSSP(bounds=PermutationVar(valid_set=[0, 1, 2, 3, 4], name="per"),
                  processing_times=processing_times,
                  minmax="min")


# Define the model and solve the problem
# model = SA.OriginalSA(epoch=1000, pop_size=50, temp_init = 100, step_size = 0.1)
# model.solve(problem=problem_jssp)

# FIXME: seems like tabu search is bugged
# model = TS.OriginalTS(epoch=1000, pop_size=50, tabu_size=5, neighbour_size=20, perturbation_scale=0.05)
# model.solve(problem=problem_jssp)

# model = GA.BaseGA(epoch=1000, pop_size=50, pc=0.9, pm=0.05)
# model.solve(problem=problem_jssp)

model = ACOR.OriginalACOR(epoch=1000, pop_size=50, sample_count = 25, intent_factor = 0.5, zeta = 1.0)
model.solve(problem=problem_jssp)

2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: Solving single objective optimization problem.
2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: >>>Problem: P, Epoch: 1, Current best: 825.0, Global best: 825.0, Runtime: 0.01506 seconds
2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: >>>Problem: P, Epoch: 2, Current best: 825.0, Global best: 825.0, Runtime: 0.01694 seconds
2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: >>>Problem: P, Epoch: 3, Current best: 825.0, Global best: 825.0, Runtime: 0.01322 seconds
2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: >>>Problem: P, Epoch: 4, Current best: 825.0, Global best: 825.0, Runtime: 0.01710 seconds
2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: >>>Problem: P, Epoch: 5, Current best: 822.0, Global best: 822.0, Runtime: 0.01346 seconds
2024/05/20 12:46:29 PM, INFO, mealpy.swarm_based.ACOR.OriginalACOR: >>>Problem: P, Epoch: 

id: 27056, target: Objectives: [822], Fitness: 822.0, solution: [0 1 3 2 4]